# From Scratch: Building a Neural Network with NumPy

This notebook contains an end-to-end implementation of a simple feedforward neural network using **only NumPy** — no deep learning frameworks involved. It aims to demystify the inner workings of neural networks by walking through each component step-by-step, with a strong focus on **clarity, interactivity, and visualization**.

## Key Features

- **Manual forward and backward passes** (no autograd)
- Training on toy datasets (e.g., digit recognition or synthetic classification)
- Loss and accuracy plots updated in real-time
- Visual explanation of gradients and weight updates
- Interactive sliders or inputs (if supported) to adjust hyperparameters
- Clean, well-commented code intended for learning and experimentation

> *This project is inspired by university coursework, but developed independently from scratch to reinforce my understanding and extend the ideas further.*

In case of any questions or comments, please contact me at ea.arseneva@gmail.com


In [ ]:
import numpy as np
from abc import ABC, abstractmethod

In [ ]:
#Define abstract classes for Layer, Loss, Optimizer

class Layer(ABC):
    
    def __init__(self, input_dim=None, output_dim=None):
        self._input_dim = input_dim
        self._output_dim = output_dim
        self.input = None

    @property
    def input_dim(self):
        return self._input_dim

    @property
    def output_dim(self):
        return self._output_dim
    
    
    @abstractmethod
    def forward(self, x: np.ndarray) -> np.ndarray:
        """
        Forward pass through the layer.
        Args:
            x (np.ndarray): Input data. The shape should be (batch_size, input_dim).
        Returns:
            np.ndarray: Output data. The shape should be (batch_size, output_dim).
        """
        pass

    @abstractmethod
    def backward(self, grad: np.ndarray) -> np.ndarray:
        """
        Backward pass through the layer.
        Args:
            grad (np.ndarray): Gradient of the loss with respect to the output. 
            The shape should be (batch_size, output_dim).
        Returns:
            np.ndarray: Gradient of the loss with respect to the input.
            The shape should be (batch_size, input_dim).
        """
        pass

class Loss(ABC):
    @abstractmethod
    def forward(self, y_true: np.ndarray, y_pred: np.ndarray) -> float:
        """
        Forward pass through the loss function.
        Args:
            y_true (np.ndarray): True labels. The shape should be (batch_size,).
            y_pred (np.ndarray): Predicted labels. The shape should be (batch_size,).
        Returns:
            float: Computed loss. The shape should be a scalar.
        """
        pass

    @abstractmethod
    def backward(self, y_true: np.ndarray, y_pred: np.ndarray) -> np.ndarray:
        """
        Backward pass through the loss function.
        Args:
            y_true (np.ndarray): True labels. The shape should be (batch_size,).
            y_pred (np.ndarray): Predicted labels. The shape should be (batch_size,).
        Returns:
            np.ndarray: Gradient of the loss with respect to the predictions. The shape should be (batch_size,).
        """
        pass
    
class Optimizer(ABC):
    @abstractmethod
    def step(self, params: np.ndarray, grads: np.ndarray) -> None:
        """
        Update the parameters based on the gradients.
        Args:
            params (np.ndarray): Parameters to be updated. The shape should be (num_params,).
            grads (np.ndarray): Gradients of the loss with respect to the parameters. The shape should be (num_params,).
        Returns:
            None
        """
        pass
    

In [ ]:
# Define concrete implementation of Layer: Linear, ReLU, SoftMax, and Sequential

class ReLU(Layer):
    def __init__(self):
        super().__init__()
        
    def forward(self, x: np.ndarray) -> np.ndarray:
        self.input = x
        return np.maximum(0, x)

    def backward(self, grad: np.ndarray) -> np.ndarray:
        assert grad.shape == self.input.shape, f"Gradient shape {grad.shape} does not match input shape {self.input.shape}"
        # Gradient of ReLU is 1 for positive inputs, 0 for negative inputs
        return grad * (self.input > 0)
    
    
class Linear(Layer):
    def __init__(self, input_dim: int, output_dim: int):
        assert input_dim > 0 and output_dim > 0, "Input and output dimensions of a Linear layer must be positive integers."
        super().__init__(input_dim, output_dim)
        # Initialize weights and bias
        self.weights = np.random.randn(input_dim, output_dim) * 0.01
        self.bias = np.zeros((1, output_dim))
        self.grad_weights = None
        self.grad_bias = None   

    def forward(self, x: np.ndarray) -> np.ndarray:
        assert x.shape[1] == self.weights.shape[0], f"Input shape {x.shape} does not match expected shape (batch_size, {self.weights.shape[0]})"
        assert self.weights.shape[1] == self.bias.shape[1], f"Weights shape {self.weights.shape} does not match bias shape {self.bias.shape}"
        self.input = x
        return x @ self.weights + self.bias

    def backward(self, grad: np.ndarray) -> np.ndarray:
        assert grad.shape[1] == self.bias.shape[1], f"Gradient shape {grad.shape} does not match bias shape {self.bias.shape}"
        assert grad.shape[0] == self.input.shape[0], f"Gradient shape {grad.shape} does not match input shape {self.input.shape}"
        # Gradient of the loss with respect to the input
        grad_input = grad @ self.weights.T
        # Gradient of the loss with respect to the weights and bias
        self.grad_weights = self.input.T @ grad
        self.grad_bias = np.sum(grad, axis=0, keepdims=True)
        return grad_input
    
class SoftMax(Layer):
    def __init__(self):
        super().__init__()
        self.output = None
        
    def forward(self, x: np.ndarray) -> np.ndarray:
        # Subtract max for numerical stability
        exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
        self.output = exp_x / np.sum(exp_x, axis=1, keepdims=True)
        return self.output

    def backward(self, grad: np.ndarray) -> np.ndarray:
        assert grad.shape == self.output.shape, f"Gradient shape {grad.shape} does not match output shape {self.output.shape}"
        # Gradient of the loss with respect to the input
        return grad * self.output * (1 - self.output)
    
class Sequential(Layer):
    def __init__(self, layers: list):
        self.layers = layers
        super().__init__(layers[0].input_dim, layers[-2].output_dim)
        self.__check_consistency__()

    def __check_consistency__(self):
        assert len(self.layers) > 1, "Sequential model must have at least one layer."
        assert self.layers[0].input_dim is not None, "First layer input dimension must be specified."
        # here we assume the last layer is a  softmax layer, so we check the second last layer
        assert self.layers[-2].output_dim is not None, "Last layer output dimension must be specified."
        assert self.layers[0].input_dim == self.input_dim, f"First layer input dimension {self.layers[0].input_dim} does not match expected input dimension {self.input_dim}"
        assert self.layers[-2].output_dim == self.output_dim, f"Last layer output dimension {self.layers[-1].output_dim} does not match expected output dimension {self.output_dim}"
        current_dim = self.input_dim
        mismatch_list = []
        for layer in self.layers:
            if layer.input_dim != None:
                if layer.input_dim != current_dim: 
                    mismatch_list.append(f"Layer {layer.__class__.__name__} input dimension {layer.input_dim} does not match expected input dimension {current_dim}")
                current_dim = layer.output_dim
       # if current_dim != self.layers[-2].output_dim: 
       #     mismatch_list.append(f"Last layer output dimension {self.layers[-2].output_dim} does not match expected output dimension {current_dim}")
        assert len(mismatch_list) == 0, f"Layer dimension mismatch: {'\n'.join(mismatch_list)}"
            
             
    def forward(self, x: np.ndarray) -> np.ndarray:
        assert x.shape[1] == self.layers[0].input_dim, f"Input shape {x.shape} does not match expected shape (batch_size, {self.layers[0].input_dim})"
        for layer in self.layers:
            x = layer.forward(x)
        return x

    def backward(self, grad: np.ndarray) -> np.ndarray:
        for layer in reversed(self.layers):
            grad = layer.backward(grad)
        return grad

In [ ]:
# Define concrete implementation of Loss: MeanSquaredError and CrossEntropy

class MeanSquaredError(Loss):
    def forward(self, y_true: np.ndarray, y_pred: np.ndarray) -> float:
        assert y_true.shape == y_pred.shape, f"True labels shape {y_true.shape} does not match predicted labels shape {y_pred.shape}"
        return np.mean(np.square(y_true - y_pred))

    def backward(self, y_true: np.ndarray, y_pred: np.ndarray) -> np.ndarray:
        assert y_true.shape == y_pred.shape, f"True labels shape {y_true.shape} does not match predicted labels shape {y_pred.shape}"
        return 2 * (y_pred - y_true) / y_true.size
    
    
class CrossEntropy(Loss):
    def forward(self, y_true: np.ndarray, y_pred: np.ndarray) -> float:
        assert y_true.shape == y_pred.shape, f"True labels shape {y_true.shape} does not match predicted labels shape {y_pred.shape}"
        # Clip predictions to prevent log(0)
        y_pred = np.clip(y_pred, 1e-15, 1 - 1e-15)
        return -np.mean((y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred)), axis=1)

    def backward(self, y_true: np.ndarray, y_pred: np.ndarray) -> np.ndarray:
        assert y_true.shape == y_pred.shape, f"True labels shape {y_true.shape} does not match predicted labels shape {y_pred.shape}"
        # Clip predictions to prevent division by zero
        y_pred = np.clip(y_pred, 1e-15, 1 - 1e-15)
        return (y_pred - y_true) / (y_pred * (1 - y_pred))

In [ ]:
# Define concrete implementation of Optimizer: SGD

class SGD(Optimizer):
    def __init__(self, learning_rate: float = 0.01):
        assert learning_rate > 0, "Learning rate must be a positive number."
        self.learning_rate = learning_rate

    def step(self, params: np.ndarray, grads: np.ndarray) -> None:
        assert params.shape == grads.shape, f"Parameters shape {params.shape} does not match gradients shape {grads.shape}"
        params -= self.learning_rate * grads

In [ ]:
# Put everything together in a training loop
def train(model: Sequential, loss_fn: Loss, optimizer: Optimizer, x_train: np.ndarray, y_train: np.ndarray, batch_size: int = 32, epochs: int = 1000):
    for epoch in range(epochs):
        # go in batches
        for i in range(0, x_train.shape[0], batch_size):
            x_batch = x_train[i:i + batch_size]
            y_batch = y_train[i:i + batch_size]
            
            # Forward pass
            y_pred = model.forward(x_batch)
            
            # Compute loss
            loss = loss_fn.forward(y_batch, y_pred)
            
            # Backward pass
            grad = loss_fn.backward(y_batch, y_pred)
            model.backward(grad)
            
            # Update parameters
            for layer in model.layers:
                if isinstance(layer, Linear):
                    optimizer.step(layer.weights, layer.grad_weights)
                    optimizer.step(layer.bias, layer.grad_bias)

        # Print loss every 100 epochs
        if epoch % 100 == 0:
            print(f"Epoch {epoch}, Loss: {loss}")
            


In [ ]:
#Define a simple model and apply it to spiral dataset
def generate_spiral_data(n_points_per_class: int, n_classes: int):
    x = []
    y = []
    for j in range(n_classes):
        ix = range(n_points_per_class * j, n_points_per_class * (j + 1))
        r = np.linspace(0.0, 1, n_points_per_class)
        t = np.linspace(j * 4, (j + 1) * 4, n_points_per_class) + np.random.randn(n_points_per_class) * 0.2
        x1 = r * np.sin(t)
        x2 = r * np.cos(t)
        x.append(np.c_[x1, x2])
        y.append(np.full(n_points_per_class, j))
    x = np.vstack(x)
    y = np.hstack(y)
    y_one_hot = np.eye(n_classes)[y]
    return x, y_one_hot


x_train, y_train = generate_spiral_data(1000, 3)
x_test, y_test = generate_spiral_data(200, 3)

model = Sequential([
    Linear(2, 64),
    ReLU(),
    Linear(64, 3),
    SoftMax()
])
loss_fn = CrossEntropy()
optimizer = SGD(learning_rate=0.01)
batch_size = 32
# Train the model
train(model, loss_fn, optimizer, x_train, y_train, epochs=1000)
# Test the model
y_pred = model.forward(x_test)
predicted_labels = np.argmax(y_pred, axis=1)
true_labels = np.argmax(y_test, axis=1)
accuracy = np.mean(predicted_labels == true_labels)
print(f"Test Accuracy: {accuracy * 100:.2f}%")